In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [26]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG19_opt/run1.h5';

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    with tf.device('/device:GPU:0'):
        base_model = VGG19(weights = 'imagenet',
                           include_top=False,
                           input_shape=(im_size, im_size, 3))

        # Add a new top layer
        x = base_model.output
        x = Flatten()(x)
        x = Dropout({{uniform(0,1)}})(x)
        x = Dense(1024,activation='relu')(x)
        x = Dense(512,activation='relu')(x)
        predictions = Dense(num_class, activation='softmax')(x)

        # This is the model we will train
        model = Model(inputs=base_model.input, outputs=predictions)

        # First: train only the top layers (which were randomly initialized)
        for i in range(len(base_model.layers)):
            base_model.layers[i].trainable = False

        model.compile(loss='categorical_crossentropy', 
                      optimizer={{choice(['adam','rmsprop','sgd'])}}, 
                      metrics=['accuracy'])

        callbacks_list = [];
        callbacks_list.append(keras.callbacks.EarlyStopping(
            monitor='val_acc',
            patience=10,
            verbose=1));
        
        print('Tensorboard NOT activated')
        
        model.fit(X_train, Y_train,
          epochs=100,
          batch_size = {{choice([16,32,64,128,256])}},
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)
        
        score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
        print('Test accuracy:', acc)
        return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='VGG19Optimization_1')

X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg19 import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
    pass

try:
    from subprocess import check_output
except:
    pass

>>> Hyperas se

100%|██████████| 10222/10222 [00:36<00:00, 283.00it/s]


Splitting into training/validation
Creating model
Tensorboard NOT activated
Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 41s 6ms/step - loss: 4.8905 - acc: 0.0082 - val_loss: 4.7814 - val_acc: 0.0140
Epoch 2/100
7155/7155 [==============================] - 29s 4ms/step - loss: 4.8106 - acc: 0.0137 - val_loss: 4.7736 - val_acc: 0.0153
Epoch 3/100
7155/7155 [==============================] - 29s 4ms/step - loss: 4.7830 - acc: 0.0126 - val_loss: 4.7597 - val_acc: 0.0196
Epoch 4/100
7155/7155 [==============================] - 29s 4ms/step - loss: 4.7673 - acc: 0.0150 - val_loss: 4.7481 - val_acc: 0.0218
Epoch 5/100
7155/7155 [==============================] - 29s 4ms/step - loss: 4.7517 - acc: 0.0171 - val_loss: 4.7322 - val_acc: 0.0228
Epoch 6/100
7155/7155 [==============================] - 29s 4ms/step - loss: 4.7262 - acc: 0.0225 - val_loss: 4.7025 - val_acc: 0.0342
Epoch 7/100
7155/7155 [==============================] - 29s

Epoch 60/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4901 - acc: 0.1894 - val_loss: 3.8460 - val_acc: 0.1399
Epoch 61/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4916 - acc: 0.1873 - val_loss: 3.8912 - val_acc: 0.1259
Epoch 62/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4649 - acc: 0.1874 - val_loss: 3.8498 - val_acc: 0.1334
Epoch 63/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4517 - acc: 0.1936 - val_loss: 3.8757 - val_acc: 0.1356
Epoch 64/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4528 - acc: 0.1944 - val_loss: 3.8083 - val_acc: 0.1402
Epoch 65/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4442 - acc: 0.1883 - val_loss: 3.8208 - val_acc: 0.1392
Epoch 66/100
7155/7155 [==============================] - 29s 4ms/step - loss: 3.4331 - acc: 0.1950 - val_loss: 3.8112 - val_acc: 0.1386
Epoch 67/100
7155/7155 [=================

Epoch 15/100
7155/7155 [==============================] - 27s 4ms/step - loss: 1.1246 - acc: 0.6932 - val_loss: 4.9797 - val_acc: 0.1275
Epoch 16/100
7155/7155 [==============================] - 26s 4ms/step - loss: 0.9857 - acc: 0.7205 - val_loss: 5.3360 - val_acc: 0.1206
Epoch 17/100
7155/7155 [==============================] - 26s 4ms/step - loss: 0.8636 - acc: 0.7530 - val_loss: 5.4551 - val_acc: 0.1184
Epoch 18/100
7155/7155 [==============================] - 27s 4ms/step - loss: 0.7426 - acc: 0.7866 - val_loss: 5.6273 - val_acc: 0.1174
Epoch 19/100
7155/7155 [==============================] - 27s 4ms/step - loss: 0.6514 - acc: 0.8131 - val_loss: 5.7840 - val_acc: 0.1187
Epoch 20/100
7155/7155 [==============================] - 27s 4ms/step - loss: 0.5762 - acc: 0.8306 - val_loss: 5.8695 - val_acc: 0.1242
Epoch 21/100
7155/7155 [==============================] - 26s 4ms/step - loss: 0.5100 - acc: 0.8485 - val_loss: 6.2124 - val_acc: 0.1200
Epoch 22/100
7155/7155 [=================

Epoch 00021: early stopping
Test accuracy: 0.135311379203
Getting data


100%|██████████| 10222/10222 [00:36<00:00, 281.34it/s]


Splitting into training/validation
Evalutation of best performing model:
3067/3067 [==============================] - 9s 3ms/step
[3.7964961458037645, 0.14150635801185255]


In [28]:
best_model.save(modelPath);
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 22, 22, 128)       0         
__________

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [18]:
# preds = model.predict(x_test, verbose=1)

In [19]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)